In [1]:
import pandas as pd

In [2]:
ru_y2_metadata = pd.read_csv('HPRC_RU_Y2_Sequel_Metadata_PacBio_HiFi_Submission.csv')
ru_y2_metadata['flow_cell'] = [flow_cell.split('.')[0] for flow_cell in ru_y2_metadata['filename'].tolist()]
ru_y2_metadata.shape

(54, 24)

In [3]:
ru_y2_qc = pd.read_csv('RU_Y2_HIFI_post_sra_metadata.csv')
ru_y2_qc.rename(columns={'filename':'sample_ID_filename', 'bam_file':'filename'},inplace=True)
print(ru_y2_qc.shape)


(36, 20)


In [4]:
# 18 files are not present in qc sample: HG04187
ru_y2_metadata[~ru_y2_metadata['flow_cell'].isin(ru_y2_qc['flow_cell'].tolist())].shape 

(18, 24)

In [5]:
# all flow cells are present between qc and metadata
ru_y2_qc = ru_y2_qc[ru_y2_metadata['flow_cell'].isin(ru_y2_metadata['flow_cell'].tolist())]

/data/tmp/ipykernel_3688859/2943073831.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ru_y2_qc = ru_y2_qc[ru_y2_metadata['flow_cell'].isin(ru_y2_metadata['flow_cell'].tolist())]


In [6]:
ru_y2_df = pd.merge( ru_y2_metadata, ru_y2_qc[['flow_cell','filename']], on='flow_cell', how='inner')
ru_y2_df.drop('filename_x', axis=1, inplace=True)
ru_y2_df.rename(columns={'filename_y':'filename'}, inplace=True)

In [7]:
ru_y2_setup = pd.read_csv('RU_Y2_HIFI.csv', header=None)

In [8]:
ru_y2_setup = ru_y2_setup[ru_y2_setup[0].str.contains('primrose')]
ru_y2_setup['filename'] = [filename.split('/')[-1] for filename in ru_y2_setup[0].tolist()]
ru_y2_setup.rename(columns={0:'s3-submission'},inplace=True)

In [9]:
# ru_y2_setup
# RU_Y2_HiFi.transfer_ec2.csv
# s3://human-pangenomics/working/HPRC

In [10]:
transfer_list = []
for data in ru_y2_setup['s3-submission'].tolist():
	path = data.split('/746FF75B-3C32-42AF-92BB-290BA92CF89A--RU_Y2_HIFI')[1].split('primrose')
	path.insert(1, 'raw_data/PacBio_HiFi')
	path = ''.join(path)
	release = 's3://human-pangenomics/working/HPRC' + path
	transfer_list.append(release)
ru_y2_setup['s3-working'] = transfer_list
ru_y2_setup[['s3-submission','s3-working']].to_csv('RU_Y2_HiFi.transfer_ec2.csv',index=False, header=None)


In [11]:
ru_y2_df = pd.merge(ru_y2_df,ru_y2_setup, on='filename', how='inner')

In [13]:
ru_y2_df.to_csv('HPRC_RU_Y2_Sequel_Metadata_PacBio_HiFi_Submission_Final.csv',index=False)